In [1]:
import urllib.request
from bs4 import BeautifulSoup

In [2]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
url = "http://www.metacritic.com/game/pc/dishonored-2/user-reviews"
headers={'User-Agent':user_agent,} 

request = urllib.request.Request(url,None,headers) #The assembled request
data = urllib.request.urlopen(request).read()

In [13]:
soup = BeautifulSoup(data,"lxml")
print(soup.prettify()[50000:100000])

            </div>
                             </li>
                             <li class="rating_value_item rating_value_30">
                              <div class="rating_action rating_action_game rating_action_30 " data-rateref="/jl/rate/set-rating?score=3&amp;ref_type_id=30&amp;ref_id=480896" rel="nofollow" title="Rate 3 out of 10">
                               <span class="rating_value">
                                3
                               </span>
                              </div>
                             </li>
                             <li class="rating_value_item rating_value_20">
                              <div class="rating_action rating_action_game rating_action_20 " data-rateref="/jl/rate/set-rating?score=2&amp;ref_type_id=30&amp;ref_id=480896" rel="nofollow" title="Rate 2 out of 10">
                               <span class="rating_value">
                                2
                               </span>
                            

Stuff of interest: <br>
div class="review_body" - the full review is here <br>
span class="blurb blurb_expanded" - if the review is long, the full review is here <br>
div class="review_grade" - 1-5 <br>
div class="date" - date of rating <br>
div class="name" - rater's name <br>

In [4]:
# get all the data associated with each review, where each element is a diff review
reviews = soup.findAll('li', attrs={'class': 'review user_review'})

In [9]:
# an example of a review where the text is in "span class
reviews[0]

<li class="review user_review" id="user_review_6803714">
<div class="review_btm review_btm_r"><div class="review_btm review_btm_l"><div class="review_top review_top_r"><div class="review_top review_top_l">
<div class="review_content">
<div class="review_section">
<div class="review_stats">
<div class="review_critic">
<div class="name">
<a href="/user/GoldieFoxx">GoldieFoxx</a>
</div>
<div class="date">Nov 10, 2016</div>
</div>
<div class="review_grade">
<div class="metascore_w user medium game mixed indiv">5</div>
</div>
</div>
<div class="review_body">
<span class="inline_expand_collapse inline_collapsed" id="review_blurb_6803714"><span class="blurb blurb_collapsed">This game gives me very mixed feelings. The story, graphics and gameplay are all fantastic and a great step up from the original.
<br/> <br/>It has a fantastic feel to it and if you enjoyed the original you will certainly enjoy this one.
If that was it, I would have easily given this a 9/10

BUT

There are so many things d

In [10]:
review_text = []
for review in reviews:
    review_text.append(review.find('span', attrs={'class': 'blurb blurb_expanded'}))

TypeError: 'NoneType' object is not callable

42


In [17]:
print(len(reviews))

71
